# Video Classification on CM

* Here we generate real-time classical music instruments information generator on classical music concert video input. 
* We basically adopt video classification method using pytorch, to classify what instrument is currently viewed on the screen. 
* Audience will receive the streaming video with information on it.

- - -

In [1]:
# importing required components 
import numpy as np
import pandas as pd
import torch
import os
import time
import json
import urllib
import random
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
from torch.optim import lr_scheduler
from sklearn.model_selection import train_test_split
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from PIL import Image
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from efficientnet_pytorch import EfficientNet

In [3]:
# # importing remaining components
# import json
# import urllib
# from pytorchvideo.data.encoded_video import EncodedVideo

# from torchvision.transforms import Compose, Lambda
# from torchvision.transforms._transforms_video import (
#     CenterCropVideo,
#     NormalizeVideo,
# )
# from pytorchvideo.transforms import (
#     ApplyTransformToKey,
#     ShortSideScale,
#     UniformTemporalSubsample
# )

In [4]:
class Args:
    # arugments
    epochs=20
    bs=6
    lr=0.001
    momentum=0.9
    num_channels=3  # due to RGB channels(image)
    num_classes=17  # total 17 instruments labeled
    verbose='store_true'
    seed=710674

args = Args()    

np.random.seed(args.seed)
random.seed(args.seed)
torch.manual_seed(args.seed)

In [5]:
#Setting torch environment

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
    
print('Using PyTorch version:', torch.__version__, ' Device: ', DEVICE)

Using PyTorch version: 1.7.1  Device:  cuda


## Data Crawling for model training

* Here we create each folder for classical instruments that used in classical music concert
* bassdrum / bassoon / cello / clarinet / contrabass / cymbales / flute / horn / oboe / snaredrum / percussion / tamtam / timpani / trombone / trumpet / tuba / viola / violin -- total 17 instruments

### Image extraction from classical music concert video

- - -

In [6]:
import cv2
from glob import glob

We are using orchestra symphony video as validation

In [ ]:
# ## Code for 
# def create_dir(path):
#     try:
#         if not os.path.exists(path):
#             os.makedirs(path)
#     except OSError:
#         print(f"ERROR: creating directory with name {path}")

# def save_frame(video_path, save_dir, gap=10):
#     name = video_path.split("/")[-1].split(".")[0]
#     save_path = os.path.join(save_dir, name)
#     create_dir(save_path)

#     cap = cv2.VideoCapture(video_path)
#     idx = 0

#     while True:
#         ret, frame = cap.read()

#         if ret == False:
#             cap.release()
#             break

#         if idx == 0:
#             cv2.imwrite(f"{save_path}/{idx}_Saint.png", frame)
#         else:
#             if idx % gap == 0:
#                 cv2.imwrite(f"{save_path}/{idx}_Saint.png", frame)

#         idx += 1

Extracting images

In [ ]:
# if __name__ == "__main__":
#     video_paths = glob("E:/RESEARCH/Datasets/VC/classic/videos/INPROCESS/*")
#     save_dir = "save"

#     for path in video_paths:
#         save_frame(path, save_dir, gap=50)

In [ ]:
# ## you have to end all windows before next step
# cv2.destroyAllWindows()

- - -

* Changing file names in directory

In [39]:
# file_path = "E:/RESEARCH/Datasets/VC/classic/train/woodwinds"
# file_names = os.listdir(file_path)

In [40]:
# i = 1
# for name in file_names:
#     src = os.path.join(file_path, name)
#     dst = str(i) + '.png'
#     dst = os.path.join(file_path, dst)
#     os.rename(src, dst)
#     i += 1

* Image Crop

In [ ]:
# im = Image.open(r"./test2.png")
 
# # Setting the points for cropped image
# left = 1000
# top = 200
# right =3000
# bottom = 3000
 
# # Cropped image of above dimension
# # (It will not change original image)
# im1 = im.crop((left, top, right, bottom))
 
# # Shows the image in image viewer
# im1.show()

- - -

## Training model preparation

* Selected Classical convert video: 
https://www.youtube.com/watch?v=65nvqmVhZ3g

* Our approach is train the model from image classification task, and apply it for the video classification. 
* So it is also possible to use pretrained models, such as, resnet, efficientnet

In [ ]:
# Choose the `slowFAST_r50` pretrained model - for our video classification model training 
## slowfast net is for video classification. 
model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)

In [41]:
# model_res = models.resnet18(num_classes=2, pretrained=True)
model_eff3 = EfficientNet.from_pretrained('efficientnet-b3', num_classes=args.num_classes)
# model = model_res.to(DEVICE)
model = model_eff3.to(DEVICE)

Loaded pretrained weights for efficientnet-b3


## Image classification approach

* Generating classification model with image classification task.
* Then we can adop the model for video input, to figure out the instrument on the screen.

In [42]:
# Data Transformation
data_transforms = transforms.Compose([
#     transforms.CenterCrop(1024),
    transforms.Resize(256),
    transforms.RandomResizedCrop(256),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(contrast=(0.3, 1), saturation=(0.3, 1)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456,0.406], [0.229, 0.224, 0.225])
])

In [43]:
# Uploading image data
classic_data = datasets.ImageFolder(root = 'E:/RESEARCH/Datasets/VC/classic/train', transform = data_transforms)
# sports_data = datasets.ImageFolder(root = 'E:/RESEARCH/Datasets/VC/sports/data/train', transform = data_transforms)

In [44]:
train_size = int(0.8 * len(classic_data))
test_size = len(classic_data)-train_size
print(train_size)
print(test_size)

6800
1700


In [45]:
train_dataset, test_dataset = torch.utils.data.random_split(classic_data, [train_size, test_size])

In [46]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.bs, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args.bs, shuffle=False, num_workers=4)

In [47]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
print(labels)

tensor([ 6, 14,  3, 13, 16, 10])


In [48]:
# Setting Optimizer and Objective Function

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = args.lr)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, total_steps=20, anneal_strategy='cos')

# print(model)

In [49]:
# Function for checking model performance during CNN model

def train(model, train_loader, optimizer, log_interval):
    model.train()
    print(optimizer.param_groups[0]['lr'])
    
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image), 
                len(train_loader.dataset), 100. * batch_idx / len(train_loader), 
                loss.item()))

    scheduler.step() #for learning rate scheduler

In [50]:
# Function for checking model performance during the learning process

def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    validation =[]

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
    
    test_loss /= (len(test_loader)) 
    validation_accuracy = 100. * correct / len(test_loader.dataset)
    validation.append(validation_accuracy)
    
    return test_loss, validation_accuracy

In [51]:
# Checking train, val loss and accuracy

total = []

for epoch in range(1, args.epochs):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, validation_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tValidation Accuracy: {:.2f} % \n".format(
        epoch, test_loss, validation_accuracy))
    
    total.append((test_loss, validation_accuracy))

0.0003999999999999993
Train Epoch: 1 [0/6800 (0%)]	Train Loss: 2.840494
Train Epoch: 1 [1200/6800 (18%)]	Train Loss: 2.009555
Train Epoch: 1 [2400/6800 (35%)]	Train Loss: 1.644905
Train Epoch: 1 [3600/6800 (53%)]	Train Loss: 0.612845
Train Epoch: 1 [4800/6800 (71%)]	Train Loss: 0.713958
Train Epoch: 1 [6000/6800 (88%)]	Train Loss: 0.455515

[EPOCH: 1], 	Test Loss: 0.6544, 	Validation Accuracy: 81.41 % 

0.0013167184270002508
Train Epoch: 2 [0/6800 (0%)]	Train Loss: 1.611465
Train Epoch: 2 [1200/6800 (18%)]	Train Loss: 0.918972
Train Epoch: 2 [2400/6800 (35%)]	Train Loss: 1.215746
Train Epoch: 2 [3600/6800 (53%)]	Train Loss: 1.096439
Train Epoch: 2 [4800/6800 (71%)]	Train Loss: 1.714181
Train Epoch: 2 [6000/6800 (88%)]	Train Loss: 0.773239

[EPOCH: 2], 	Test Loss: 1.7953, 	Validation Accuracy: 59.00 % 

0.003716718427000252
Train Epoch: 3 [0/6800 (0%)]	Train Loss: 0.137760
Train Epoch: 3 [1200/6800 (18%)]	Train Loss: 2.206374
Train Epoch: 3 [2400/6800 (35%)]	Train Loss: 1.721750
Train E

## Classification model test

In [54]:
# Saving pytorch model

torch.save(model.state_dict(), 'C:/Users/user/Jupyter/ML_practices/vc_classic.pt')

In [58]:
model_test = torch.load('C:/Users/user/Jupyter/ML_practices/vc_classic.pt')
print(model_test)

OrderedDict([('_conv_stem.weight', tensor([[[[ 3.6015e-01, -9.9441e-01, -4.5802e-01],
          [ 5.5020e-01, -7.6584e-01, -2.7294e-01],
          [-1.1673e-01, -9.2886e-02, -8.5131e-03]],

         [[ 1.4724e+00, -1.9369e+00, -4.5302e-01],
          [ 1.3037e+00, -1.6004e+00, -2.2481e-01],
          [ 8.3351e-02,  1.3832e-02,  8.2494e-02]],

         [[ 2.5972e-01, -7.0418e-01, -3.9598e-01],
          [ 4.1036e-01, -5.2578e-01, -1.2826e-01],
          [-1.6106e-02,  1.0031e-02,  7.0970e-02]]],


        [[[-7.3896e-01, -9.4561e-02,  3.7037e-01],
          [-2.1739e+00, -1.6889e-01,  3.9888e-01],
          [-9.0149e-01, -3.0675e-01,  2.9065e-01]],

         [[-1.5667e-01,  3.7856e-01,  9.2244e-01],
          [-9.2168e-01,  4.7206e-01,  1.0069e+00],
          [-3.4290e-01,  1.7309e-01,  8.2615e-01]],

         [[-7.7972e-01, -2.6779e-02,  5.0847e-01],
          [-1.0192e+00, -5.6711e-02,  5.6332e-01],
          [-9.3994e-01, -3.8865e-01,  3.8882e-01]]],


        [[[ 1.5129e+00,  1.9749

* Model performance test process

In [59]:
# Setting Optimizer and Objective Function

criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr = args.lr, momentum = args.momentum)
optimizer = torch.optim.Adam(model.parameters(), lr = args.lr)
# scheduler = optim.lr_scheduler.LambdaLR(optimizer = optimizer,
#                                        lr_lambda = lambda epoch:0.95 ** epoch,
#                                        last_epoch = -1,
#                                        verbose = False)

# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.1, 
#                                                 steps_per_epoch=10, epochs=10,anneal_strategy='linear')

scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, total_steps=10,anneal_strategy='cos')

# print(model)

In [ ]:
data_folder_path = 'E:/RESEARCH/Datasets/VC/classic/val'
# data_folder_path = 'E:/RESEARCH/Datasets/VC/sports/data/val'
test_dataset = datasets.ImageFolder(root=data_folder_path, transform=data_transforms)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
## Model testing
total_evaluation = []

for epoch in range(1, 10):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model_test, test_dataloader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))
    
    total_evaluation.append((test_loss, test_accuracy))